# In this notebook I mapped the 3 chosen Saudi Cities
## (Al Khobar, Riyadh, Jeddah)

<img src="Images/sa_map.png"
style="width:500px;height:500px"/>

### Steps:
1. Import libraries
2. Add inputs
3. Plot street network + add building footprints
4. Add the 30 minutes walking radius isochrone

### 1. Import libraries

In [ ]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, Polygon, LineString
from descartes import PolygonPatch

### 2. Add inputs

In [ ]:
# cities 2.5km radius bounding box coordinates
Khobar = [50.1829, 26.2774, 50.2312, 26.3213] # Al Khobar
Riyadh = [46.6503, 24.6888, 46.6986, 24.7327] # Riyadh
Jeddah = [39.1038, 21.5700, 39.1524, 21.6149] # Jeddah

network_type = 'walk'
trip_times = [5,10,15,25,30] # in minutes
travel_speed = 4.5 # walking speed in km/hour
tags = {"building": True}

### 3. Plot street network + add building footprints

In [ ]:
west, south, east, north = (39.1038, 21.5700, 39.1524, 21.6149)

# get street network from the bounding box
G = ox.graph_from_bbox(north, south, east, west, network_type=network_type)

# get building footprints from the bounding box
fp = ox.geometries_from_bbox(north, south, east, west, tags=tags)

# plot street network
fig, ax = ox.plot_graph(G, figsize=(12, 12), bgcolor='w', edge_color='k',
                        node_size=0, edge_linewidth=0.3,
                        show=False, close=False)

# add building footprints
fp.plot(ax=ax, color='gray', alpha=0.5)
plt.show()

### 4. Add the 30 minutes walking radius isochrone

In [ ]:
# assign center point to analyze
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
x, y = gdf_nodes['geometry'].unary_union.centroid.xy
center_node = ox.get_nearest_node(G, (y[0], x[0]))
G = ox.project_graph(G)

# assign time attribute
meters_per_minute = travel_speed * 1000 / 60 
for u, v, k, data in G.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='Reds', start=0.3, return_hex=True)

# make the isochrone polygons
isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

# plot the network then add isochrones as colored descartes polygon patches
fig, ax = ox.plot_graph(G, figsize=(12, 12), bgcolor='w', edge_color='k', node_size=0, edge_linewidth=0.3, show=False, close=False)
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
    ax.add_patch(patch)
plt.show()